# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [4]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Robot

In [5]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if self.client_id != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return self.client_id
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)
            if (err_code != 0):
                print(f"ERROR SENDING MOTOR VELS {err_code}")
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(self.client_id, self.frame, relative_object, position, sim.simx_opmode_streaming)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

# Trajectory follower

Follows a spline trajectory through the spheres. 

In [9]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot
#disk = robot('Disc', client_id=r.client_id)
starting_vel = array([0., 0., 0.])
starting_pos = r.get_position()
def p_spline(tf, target_vel, desired_pos):
    global starting_vel
    global starting_pos
    start_time = sim.simxGetLastCmdTime(r.client_id)
    # Time interval
    time_step = 0.1
    time_steps = linspace(0, tf, round(tf / time_step))

    a3 = (2*starting_pos+(starting_vel + target_vel) * tf - 2*desired_pos)/(tf**3)
    a2 = (3*desired_pos-3*starting_pos-2*starting_vel*tf-target_vel*tf)/(tf**2)
    a1 = starting_vel
    a0 = starting_pos

    for t in time_steps:
        point_traj = a3 * t**3 + a2 * t**2 + a1 * t + a0
        vel_traj = 3 * a3 * t**2 + 2 * a2 * t + a1

        # Sensing
        robot_position = r.get_position()
        # disk.set_position(point_traj)

        # Trajectory tracker
        u = 30 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])

        if (sim.simxGetLastCmdTime(r.client_id) - start_time < (t + time_step) * 1000):
            time.sleep((t + time_step) - (sim.simxGetLastCmdTime(r.client_id) - start_time) / 1000)
        else:
            print(f"BEHIND{sim.simxGetLastCmdTime(r.client_id) - start_time - (t + time_step)*1000}")
    starting_vel = target_vel
    starting_pos = desired_pos
p_spline(17, array([0., -0.1, 0.]), r.get_object_position('Sphere1'))

p_spline(20, array([0., -0.3, 0.]), array([4.5, 6., 0.5]))

p_spline(12, array([-0.2, 0.1, 0.]), r.get_object_position('Sphere'))

p_spline(35, array([0., -0.4, 0.]), r.get_object_position('Sphere0'))

p_spline(30, array([0.4, 0.15, 0.]), r.get_object_position('Sphere2'))

p_spline(45, array([-0.3, 0., 0.]), r.get_object_position('Sphere3'))

p_spline(25, array([-0.2, 0.2, 0.]), array([0.5, -5., 0.5]))

p_spline(50, array([-0.1, -0.2, 0.]), array([-4.5, -8., 0.5]))

p_spline(5, array([0., 0., 0.]), r.get_object_position('Sphere4'))

r.send_motor_velocities([0., 0., 0., 0.])

r.close_connection() 

Robot connected
ERROR SENDING MOTOR VELS 1
ERROR SENDING MOTOR VELS 1
ERROR SENDING MOTOR VELS 1
ERROR SENDING MOTOR VELS 1
BEHIND50.0
BEHIND0.0
BEHIND0.0
Connection closed
